In [1]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
from shapely.geometry import shape as s_shape

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [8]:
path = DATA_DIR/'cairns_gtfs.zip'
feed = gt.read_gtfs(path, dist_units='km')

# Pick date
date = gt.get_first_week(feed)[0]
print('date', date)


date 20140526


In [9]:
ts = gt.compute_trip_stats(feed)
ts.head()

,trip_id,route_id,route_short_name,route_type,direction_id,shape_id,num_stops,start_time,end_time,start_stop_id,end_stop_id,is_loop,duration,distance,speed
703,CNS2014-CNS_MUL-Weekday-00-4165878,110-423,110,3,0,1100023,35,05:50:00,06:50:00,750337,750449,0,1.000000,32.507121,32.507121
0,CNS2014-CNS_MUL-Saturday-00-4165937,110-423,110,3,0,1100023,35,06:16:00,07:10:00,750337,750449,0,0.900000,32.507121,36.119024
704,CNS2014-CNS_MUL-Weekday-00-4165879,110-423,110,3,0,1100023,35,06:20:00,07:20:00,750337,750449,0,1.000000,32.507121,32.507121
705,CNS2014-CNS_MUL-Weekday-00-4165880,110-423,110,3,0,1100023,35,06:50:00,07:50:00,750337,750449,0,1.000000,32.507121,32.507121
706,CNS2014-CNS_MUL-Weekday-00-4165881,110-423,110,3,0,1100023,35,07:15:00,08:20:00,750337,750449,0,1.083333,32.507121,30.006574


In [10]:
rts = gt.compute_route_time_series(feed, ts, date)
gt.downsample(rts, freq='12H')

indicator           num_trip_starts                                           \
route_id                    110-423 111-423 112-423 113-423 120-423 120N-423   
2014-05-26 00:00:00              23      22       5       3      12        0   
2014-05-26 12:00:00              36      36      10       3      20        2   

indicator                                              ...    service_speed  \
route_id            121-423 122-423 123-423 130-423    ...          131-423   
2014-05-26 00:00:00      14      14      23      12    ...        24.049902   
2014-05-26 12:00:00      20      19      37      21    ...        24.051800   

indicator                                                                   \
route_id              131N-423    133-423    140-423    141-423    142-423   
2014-05-26 00:00:00        NaN  25.035081  25.634669  20.802088  26.021598   
2014-05-26 12:00:00  24.067374  26.439134  25.832253  20.828188  26.417789   

indicator                                                       
route_id               143-423   143W-423    150-423  150E-423  
2014-05-26 00:00:00  24.366138        NaN  31.565566       NaN  
2014-05-26 12:00:00  24.307749  30.331218  31.547064  34.12138  

[2 rows x 100 columns]

In [15]:
trip_id = feed.trips['trip_id'].iat[0]
geo = gt.trip_to_geojson(feed, trip_id, include_stops=True)
path = DATA_DIR/'trip_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [16]:
route_id = feed.routes['route_id'].iat[0]
geo = gt.route_to_geojson(feed, route_id, include_stops=True)
path = DATA_DIR/'route_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
# Add distances to feed
trips_stats = gt.compute_trip_stats(feed, compute_dist_from_shapes=True)
feed.stop_times = gt.append_dist_to_stop_times(feed, trips_stats)
feed.stop_times.T

In [ ]:
trips_stats = gt.compute_trip_stats(feed)
routes_stats = gt.compute_route_stats(feed, trips_stats, date)
routes_stats


In [ ]:
a = pd.Series([np.nan, np.nan])
a.dropna().max()
#gt.get_trips(feed, date, "07:30:00")


In [ ]:
st['departure_time'].unique

In [ ]:
tl = gt.compute_trip_locations(feed, date, ['12:00:00'])
tl

In [ ]:
freq = '12H'
f = gt.compute_route_time_series(feed, trips_stats, date, split_directions=True, freq=freq)

# cols_and_hows = [
#     ('num_trips', 'mean'),
#     ('num_trip_starts', 'sum'),
#     ('service_distance', 'sum'),
#     ('service_duration', 'sum'),
#     ]

# frames = []
# for col, how in cols_and_hows:
#     frames.append(f[col].resample(freq).agg({col: how}))    
# g = pd.concat(frames, axis=1)
# g = g.sortlevel(axis=1)

# #f = f.resample(freq).sum()

# g.columns.names = f.columns.names

# g.T
f.T

In [ ]:

# Pick screen line
path = DATA_DIR/'cairns_screen_line.geojson'
with path.open() as src:
    line = json.load(src)
    line = s_shape(line['features'][0]['geometry'])

g = gt.compute_screen_line_counts(feed, line, date)
g

In [ ]:
g.groupby(['route_id', 'orientation']).count()